## Множественная проверка гипотез

### План семинара

1. Задача множественной проверки гипотез, зачем необходимо на практике
2. Как высчитывать уровень значимости для каждой из гипотез.
3. Сравниваем значения в A/B тестировании руками и при помощи Python.

## 1. Выстраиваем pipeline статистики разными способами

Главной особенностью работы с любой задачей статистики является выдвижение огромного количества гипотез. Поэтому важно выстроить pipeline для работы с данными!


Ранее мы рассмотрели следующие подходы:

1. Проверка того, что среднее значение у выборки имеет такой-то вид (критерий Вальда)
2. Проверка того, что выборка распределения является нормальной (ks-test, критерии Андерсона-Дарлинга).
3. Проверка того, что средние значения выборок совпадают или отличны (критерии Фишера, Краскела-Уоллиса)
4. Какое соотношение между средними величинами (попарное сравнение средних)

<b>Задача.</b> Какое наименьшее тестов необходимо провести, если оказывается, что средние значения 5 выборок практически совпадают, они нормальные, но при этом сложно сравнимы между собой.

Будем рассматривать следующие случаи и метрики в работе с данными:
  - FWER - количество отвергнутых гипотез, хотя они верны
  - FDR - средняя доля отвергнутых верных гипотез к доле всех отвергнутых гипотез.

Рассмотрим шаги, которые необходимы для последовательной проверки гипотез на нормальность.


Для этого обозначим те критерии, которые мы можем использовать при проверке нормальности:
* Критерий хи-квадрат
* ks-test
* Критерий Андерсона-Дарлинга
* Критерйи Шапиро-Уилка


В чем особенность этих тестов на нормальность? Что мы делали неаккуратно в семинаре проверки на нормальность?

## 2. Различные процедуры последовательной проверки гипотез

Проведем процедуру нисходящего тестирования для проверки нормальности. Начнем с самого мощного теста - Шапиро-Уилка

In [1]:
import pandas as pd
import scipy.stats
import numpy as np


In [4]:
data = pd.read_csv('./Data/wine.data', header=None)

In [6]:
ash = data[3]

Проверяем гипотезу Шапиро-Уилка о нормальности.

In [7]:
scipy.stats.shapiro(ash)

(0.9839471578598022, 0.03868526220321655)

Проверяем гипотезу Андерсона-Дарлинга о нормальности.

In [8]:
scipy.stats.anderson(ash)

AndersonResult(statistic=0.6784320885859643, critical_values=array([0.564, 0.642, 0.77 , 0.899, 1.069]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

Проверяем ks-test.

In [9]:
scipy.stats.kstest(ash, cdf=scipy.stats.norm(*scipy.stats.norm.fit(ash)).cdf)

KstestResult(statistic=0.057886506517667, pvalue=0.5840565143781371)

Проверяем критерий хи-квадрат.

In [10]:
def check_chi_square(distribution, sample, num_bins):
    GRID = np.linspace(0, 1, num_bins + 1)
    quantiles = distribution.ppf(GRID)
    
    bins = [
        sample[
            (sample > left) & (sample < right)
        ] for left, right in zip(quantiles[:-1], quantiles[1:])
    ]
    
    bin_sizes = np.array([len(bin_) for bin_ in bins])
    frequencies = bin_sizes / len(sample)

    return scipy.stats.chisquare(f_obs=frequencies, f_exp=np.ones(num_bins) / num_bins)

In [11]:
check_chi_square(scipy.stats.norm(*scipy.stats.norm.fit(ash)), ash, 6)

Power_divergenceResult(statistic=0.033581618482514834, pvalue=0.9999891383211704)

Упорядочиваем их по p-value и проводим нисходящую процедуру.

<b>Вопрос.</b> Какие уровни доверия должны быть для каждого из случаев?

А теперь зададимся таким вопросом: имеется куча критериев для к подбору распределения.

Рассмотрим, к примеру, нормальное распределение.
  
Что такое отвергнутая верная гипотеза: распределение нормальное, хотя мы сказали, что она неверная.

Что такое отвергнутая неверная гипотеза: распределение ненормальное, и мы сказали, что распределение не является нормальным.


В действительности, иногда важно (при подборе распределения), важно, чтобы то, что принимаем за действительность (нашли нормальность в данных, когда ее нет) - для того, чтобы нарваться на ловушку.

В качестве примера рассмотрим проверку одного из столбцов на распределение.

У нас имелось 3 гипотезы:

1. Распределение площади второго этажа нормальное (p-value = 0.002)
2. Распределение площади второго этажа имеет распределение Стьюдента с определенными параметрами (p-value = 0)
3. Логарифм распределения площади второго этажа имеет распределение Стьюдента (p-value = 0.55)

Применяем восходящюю процедуру по методу Бенджамини-Иекутиели.

$$
\alpha_i = \alpha \cdot \frac{i}{m} \cdot (\sum_{j= 1}^{m} \frac{1}{j})^{-1}
$$

<b>Вопрос 1.</b> Найдите уровни значимости критериев.

<b>Вопрос 2.</b> Какие гипотезы отвергаются на уровне FDR = 0.05?

## 3. Python в последовательной проверке гипотез 

В некоторых пакетах в Python (statsmodels) есть встроенные механизмы для последовательной проверке гипотез.

Рассмотрим пример попарного сравнения средних величин в задаче A/B тестирования.

In [12]:
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [14]:
cnts = [2103, 2187, 2166, 2138, 1974]
probs = [0.4213, 0.4803, 0.4690, 0.4656, 0.4313]

In [15]:
X_arrays = [
    np.ones(cnt) for cnt in [2103, 2187, 2166, 2138, 1974]
]

groups = [
    np.ones(cnt) * i for cnt, i in zip([2103, 2187, 2166, 2138, 1974], [1, 2, 3, 4, 5])
]

In [16]:
for i in range(5):
    X_arrays[i][int(cnts[i] * probs[i]):] = 0

In [17]:
model = MultiComparison(np.concatenate(X_arrays), np.concatenate(groups))

In [18]:
print(model.allpairtest(scipy.stats.ttest_ind, 0.05))

(<class 'statsmodels.iolib.table.SimpleTable'>, (array([[-3.90690173e+00,  9.49275927e-05],
       [-3.14328740e+00,  1.68207632e-03],
       [-2.92293192e+00,  3.48586871e-03],
       [-6.63085984e-01,  5.07312992e-01],
       [ 7.59868353e-01,  4.47374449e-01],
       [ 9.69408475e-01,  3.32395658e-01],
       [ 3.17187151e+00,  1.52562115e-03],
       [ 2.11492961e-01,  8.32512643e-01],
       [ 2.42327132e+00,  1.54240587e-02],
       [ 2.20902113e+00,  2.72281363e-02]]), array([ True,  True,  True, False, False, False,  True, False, False,
       False]), array([9.49275927e-04, 1.68207632e-02, 3.48586871e-02, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.52562115e-02, 1.00000000e+00,
       1.54240587e-01, 2.72281363e-01]), 0.005116196891823743, 0.005), array([(1.0, 2.0, -3.9069, 1.000e-04, 9.000e-04,  True),
       (1.0, 3.0, -3.1433, 1.700e-03, 1.680e-02,  True),
       (1.0, 4.0, -2.9229, 3.500e-03, 3.490e-02,  True),
       (1.0, 5.0, -0.6631, 5.073e-01, 1.000e+00, 

Какие выводы можно сделать о полученных данных?

## Что умеем?

1. Выяснять, в каких целях важны восходящие и нисходящие процедуры.
2. Уточнять имеющие гипотезы в общий pipeline.
3. Как проводить последовательную проверку гипотез в Python.

## Литература

1. https://www.statsmodels.org/dev/generated/statsmodels.sandbox.stats.multicomp.MultiComparison.html - попарное сравнение моделей в Python